# Adjust the energy sclaes of the models
The absolute values of energy is different between methods. We need to reset the scale by computing reference energies

In [1]:
from ase.calculators.nwchem import NWChem
from ase.build import molecule
import torch

/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get the reference energies
Get them by computing the energy of molecules

In [2]:
nwchem = NWChem(theory='scf', basis='6-31g*', directory='nwchem')
ref_energies = {}

Get it for H from the H2 molecule

In [3]:
h2 = molecule('H2')
ref_energies['H'] = nwchem.get_potential_energy(h2) / 2
print(f'Energy per H: {ref_energies["H"]:.2f} eV')

Energy per H: -15.33 eV


[0] ARMCI Warning: Freed 1 leaked allocations


Repeat for O and C

In [4]:
ch4 = molecule('CH4')
ref_energies["C"] = nwchem.get_potential_energy(ch4) - 4 * ref_energies["H"]
print(f'Energy per C: {ref_energies["C"]:.2f} eV')

Energy per C: -1032.44 eV


[0] ARMCI Warning: Freed 1 leaked allocations


In [5]:
h2o = molecule('H2O')
ref_energies["O"] = nwchem.get_potential_energy(h2o) - 2 * ref_energies["H"]
print(f'Energy per O: {ref_energies["O"]:.2f} eV')

Energy per O: -2037.67 eV


[0] ARMCI Warning: Freed 1 leaked allocations


## Load the Network and Change the References
We'll use the best model from the previous step

In [ ]:
model = torch.load('../0_pre-training/best_model/best_model')

Adjust the reference energies

In [ ]:
for el, en in ref_energies.items():
    model.set_reference_energy(el, en)

Save it as a starting point

We also need to change the mean and standard deviation, but this is OK for now

In [ ]:
torch.save(model, 'starting-model.pth')